# PySpark Logistic Regression

Our full dataset file has around 9 million samples. When trying to run feature_generator.

Machine:
* 2018 Mac Mini - 6 core

Docker Configuration:
* 9 CPUs
* 24 GB Ram
* 3 GB swap


# References:

* https://spark.apache.org/docs/2.2.0/mllib-feature-extraction.html
* https://towardsdatascience.com/countvectorizer-hashingtf-e66f169e2d4e
* https://medium.com/@dhiraj.p.rai/logistic-regression-in-spark-ml-8a95b5f5434c
* packaging spark job - https://developerzen.com/best-practices-writing-production-grade-pyspark-jobs-cb688ac4d20f



In [37]:
import sys
sys.path.append("../..")

import pyspark
from pyspark import SparkContext, SparkFiles
from pyspark.sql import SparkSession, DataFrameReader, SQLContext
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler, Tokenizer, IDF
from pyspark.ml import Pipeline
from pyspark.sql.functions import when, lit, col
from pyspark.sql.types import NumericType


from sklearn.utils.class_weight import compute_class_weight

import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pprint import pprint
import logging

import util.pyspark_util as pyu
import util.model_wrapper as mw



log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
sns.set()
%matplotlib inline

# allow DEBUG to be set by command line
DEBUG = bool(os.environ.get("IPYNB_DEBUG", False))

N_CLASSES = 5
FEATURE_COLUMN = "review_body"

if DEBUG:
    MIN_DF = 1
    DF_PERCENTAGE = 0.001
    DATA_FILE = "/home/jupyter/dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-test1k-preprocessed.csv"
    TEST_STR="-test"
else:
    DF_PERCENTAGE = 0.001
    DATA_FILE = "/home/jupyter/dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-500k-preprocessed.csv"
    TEST_STR=""

REPORT_FILE = f"../../reports/201911-pyspark-report{TEST_STR}.csv"


spark = SparkSession.builder \
            .master("local[*]") \
            .appName("Pyspark Wrapper Test (local)") \
            .getOrCreate()




In [3]:



class Timer(object):
    
    def __init__(self, description: str):
        self.start_time = datetime.now()
        self.description = description
        
    def stop(self):
        self.end_time = datetime.now()
        self.print_duration_min()
        
    def print_duration_min(self):
        self.duration = int((self.end_time - self.start_time).total_seconds() / 60)
        print(f"{self.description} duration: {self.duration} minutes")
        
    def get_duraction_min(self):
        return self.duration






In [4]:
file_timer = Timer("file load time")
df = spark.read.csv(SparkFiles.get(DATA_FILE), 
                    header=True, 
                    inferSchema= True)
df.collect()
file_timer.stop()

file load time duration: 0 minutes


In [5]:
def build_ngrams(inputCol, min_df, n=3):
    log.info(f'Creating TFIDF using min_df: {min_df}')
    
    tokenizer = [Tokenizer(inputCol = inputCol, outputCol = "words")]
    
    ngrams = [
        NGram(n=i, inputCol="words", outputCol="{0}_grams".format(i))
        for i in range(1, n + 1)
    ]

    vectorizers = [
        CountVectorizer(minDF=min_df, inputCol="{0}_grams".format(i),
            outputCol="{0}_counts".format(i))
        for i in range(1, n + 1)
    ]

    assembler = [VectorAssembler(
        inputCols=["{0}_counts".format(i) for i in range(1, n + 1)],
        outputCol="raw_features"
    )]
    
    idf = [IDF().setInputCol("raw_features").setOutputCol("features")]
#     idf = [IDF(minDocFreq=min_df).setInputCol("raw_features").setOutputCol("features")]

    return Pipeline(stages=tokenizer + ngrams + vectorizers + assembler + idf)



pipeline_timer = Timer("pipeline time")
# calculate a reasonable min_df
min_df = max(int(df.count() * DF_PERCENTAGE), 1)

pipeline = build_ngrams(FEATURE_COLUMN, min_df)
df = pipeline.fit(df).transform(df)
pipeline_timer.stop()


INFO:__main__:Creating TFIDF using min_df: 995


pipeline time duration: 1 minutes


In [6]:
df.printSchema()
pyu.show_df(df, ["star_rating", "features"], truncate=False)

root
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 1_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 2_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 1_counts: vector (nullable = true)
 |-- 2_counts: vector (nullable = true)
 |-- 3_counts: vector (nullable = true)
 |-- raw_features: vector (nullable = true)
 |-- features: vector (nullable = true)

+-----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Split training and test data

In [7]:
split_timer = Timer("train test split")
train, test = df.randomSplit([0.9, 0.1], seed=1)
split_timer.stop()


train_size = train.count()
test_size = test.count()

print(f'Training size: {train_size} Test size: {test_size}')

train test split duration: 0 minutes
Training size: 896004 Test size: 99684


# get number of features

In [8]:
type(train.select("features").limit(1).toPandas().features.values[0])
train.select("features").limit(1).toPandas().features.values[0].size

4073

# Assign class weights to handle imbalanced classes

In [10]:


# only do this for small files - takes too long for large datasets - we will custom compute this
# if DEBUG:
cw_timer = Timer("skelarn class weight")
labels = train.select("star_rating").toPandas().astype({"star_rating": np.int8})
class_weights = compute_class_weight('balanced', 
                                                  np.arange(1, N_CLASSES+1), 
                                                  labels.star_rating.tolist())
print(f'sklearn class weights: {class_weights}')
cw_timer.stop()
    


sklearn class weights: [1.42386715 3.02163019 2.21205515 1.20022504 0.37292633]
skelarn class weight duration: 1 minutes


In [12]:

train = train.withColumn("class_weights", lit(0))
train = train.withColumn("class_weights", when(train.star_rating == 1, class_weights[0]).otherwise(train.class_weights))
train = train.withColumn("class_weights", when(train.star_rating == 2, class_weights[1]).otherwise(train.class_weights))
train = train.withColumn("class_weights", when(train.star_rating == 3, class_weights[2]).otherwise(train.class_weights))
train = train.withColumn("class_weights", when(train.star_rating == 4, class_weights[3]).otherwise(train.class_weights))
train = train.withColumn("class_weights", when(train.star_rating == 5, class_weights[4]).otherwise(train.class_weights))


pyu.show_df(train, ["star_rating", "class_weights", "features"], 20, sample=True, truncate=True)


+-----------+------------------+--------------------+
|star_rating|     class_weights|            features|
+-----------+------------------+--------------------+
|          1|1.4238671487028725|(4073,[1,45,61,72...|
|          1|1.4238671487028725|(4073,[0,6,7,24,5...|
|          1|1.4238671487028725|(4073,[0,1,2,4,13...|
|          1|1.4238671487028725|(4073,[1,4,5,43,7...|
|          1|1.4238671487028725|(4073,[10,12,35,4...|
|          1|1.4238671487028725|(4073,[0,1,4,15,2...|
|          1|1.4238671487028725|(4073,[0,1,2,13,3...|
|          1|1.4238671487028725|(4073,[43,61,108,...|
|          1|1.4238671487028725|(4073,[0,6,9,15,1...|
|          1|1.4238671487028725|(4073,[93,114,799...|
|          1|1.4238671487028725|(4073,[5,6,8,13,1...|
|          1|1.4238671487028725|(4073,[0,1,14,26,...|
|          1|1.4238671487028725|(4073,[4,6,8,10,1...|
|          1|1.4238671487028725|(4073,[0,3,7,10,2...|
|          1|1.4238671487028725|(4073,[0,1,2,19,2...|
|          1|1.4238671487028

# Train our Model

In [15]:


import importlib
import util.constants as c
importlib.reload(pyu)
importlib.reload(mw)
importlib.reload(c)

feature_count = train.select("features").limit(1).toPandas().features.values[0].size

train_timer = Timer("traing time")
lr = LogisticRegression(labelCol="star_rating", 
                        featuresCol="features", 
                        weightCol="class_weights",
                        maxIter=100)


model = pyu.PysparkModel(model = lr,
                    train_df = train,
                    test_df = test,
                    label_column = "star_rating",
                    feature_column = "features",
                    n_classes = 5,
                         pipeline = pipeline,
                         file = DATA_FILE,
                         description=f'review_body-tfidf-df_none-ngram13-{df.count()}-{feature_count}-nolda-sampling_none{TEST_STR}',
                        model_dir="../../models")

report_dict, predict_test = model.run()
train_timer.stop()

pyu.show_df(predict_test, ["star_rating", "prediction", "rawPrediction", "probability"])



INFO:util.model_wrapper:derived name: LogisticRegression
INFO:util.model_wrapper:########################################
INFO:util.model_wrapper:Running model: name: LogisticRegression
	with file: /home/jupyter/dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-1m-preprocessed.csv
	with description: review_body-tfidf-df_none-ngram13-995688-4073-nolda-sampling_none-LogisticRegression-star_rating
	status: new
INFO:util.model_wrapper:########################################
INFO:util.time_util:Start timer for: train_time_min
INFO:util.time_util:End timer for: train_time_min
INFO:util.time_util:Total time for train_time_min: 1.82
INFO:util.time_util:Start timer for: model_save_time_min
INFO:util.pyspark_util:Saving model to file: ../../models/review_body-tfidf-df_none-ngram13-995688-4073-nolda-sampling_none-LogisticRegression-star_rating.pyspark
INFO:util.pyspark_util:Saving pipeline to file: ../../models/review_body-tfidf-df_none-ngram13-995688-4073-nolda-sampling_none-LogisticRegre

root
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 1_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 2_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 1_counts: vector (nullable = true)
 |-- 2_counts: vector (nullable = true)
 |-- 3_counts: vector (nullable = true)
 |-- raw_features: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



INFO:util.time_util:End timer for: cr_time_min
INFO:util.time_util:Total time for cr_time_min: 2.71
INFO:util.model_wrapper:calculating confusion matrix...
INFO:util.time_util:Start timer for: cm_time_min
INFO:util.time_util:End timer for: cm_time_min
INFO:util.time_util:Total time for cm_time_min: 2.59


traing time duration: 9 minutes
+-----------+----------+--------------------+--------------------+
|star_rating|prediction|       rawPrediction|         probability|
+-----------+----------+--------------------+--------------------+
|          1|       1.0|[-10.091957087099...|[5.84746444455491...|
|          1|       2.0|[-10.083085280747...|[1.09334159549751...|
|          1|       1.0|[-10.090517278128...|[1.98889104851819...|
|          1|       2.0|[-10.082841452073...|[1.04122582886483...|
|          1|       1.0|[-10.080770708826...|[1.11457771854265...|
|          1|       1.0|[-10.085338365458...|[4.89190950577153...|
|          1|       1.0|[-10.086358743623...|[2.57308332421388...|
|          1|       1.0|[-10.083879619971...|[3.50538647464572...|
|          1|       1.0|[-10.083623954277...|[2.60633226101958...|
|          1|       4.0|[-10.082998265473...|[1.03107807974704...|
+-----------+----------+--------------------+--------------------+
only showing top 10 rows



In [16]:
predict_test.printSchema()

root
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 1_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 2_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 1_counts: vector (nullable = true)
 |-- 2_counts: vector (nullable = true)
 |-- 3_counts: vector (nullable = true)
 |-- raw_features: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



# Evaluate our Model

Reference:
* https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#multiclass-classification

In [26]:
report_dict

{'model_name': 'LogisticRegression',
 'description': 'review_body-tfidf-df_none-ngram13-995688-4073-nolda-sampling_none-LogisticRegression-star_rating',
 'library': 'pyspark',
 'file': '/home/jupyter/dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-1m-preprocessed.csv',
 'model_file': '../../models/review_body-tfidf-df_none-ngram13-995688-4073-nolda-sampling_none-LogisticRegression-star_rating.pyspark',
 'pipeline_file': '../../models/review_body-tfidf-df_none-ngram13-995688-4073-nolda-sampling_none-LogisticRegression-star_rating.pipeline',
 'status': 'success',
 'status_date': '2019-11-21 18:53:49',
 'classification_report': '{"1": {"precision": 0.7026024905554779, "recall": 0.7121684867394695, "f1-score": 0.7073531483307508, "support": 14102}, "2": {"precision": 0.2902851843264549, "recall": 0.3864793949683593, "f1-score": 0.3315458457464415, "support": 6479}, "3": {"precision": 0.3212204319506342, "recall": 0.41272987556436513, "f1-score": 0.3612704226709721, "support": 9081}

In [30]:
pprint(json.loads(report_dict["classification_report"]))

{'1': {'f1-score': 0.7073531483307508,
       'precision': 0.7026024905554779,
       'recall': 0.7121684867394695,
       'support': 14102},
 '2': {'f1-score': 0.3315458457464415,
       'precision': 0.2902851843264549,
       'recall': 0.3864793949683593,
       'support': 6479},
 '3': {'f1-score': 0.3612704226709721,
       'precision': 0.3212204319506342,
       'recall': 0.41272987556436513,
       'support': 9081},
 '4': {'f1-score': 0.4188213638586097,
       'precision': 0.4012675668228162,
       'recall': 0.43798123195380173,
       'support': 16624},
 '5': {'f1-score': 0.8042531564838713,
       'precision': 0.8594705118101851,
       'recall': 0.7557024607663209,
       'support': 53398},
 'accuracy': 0.6413165603306449,
 'macro avg': {'f1-score': 0.5246487874181291,
               'precision': 0.5149692370931136,
               'recall': 0.5410122899984633,
               'support': 99684},
 'weighted avg': {'f1-score': 0.6551891250685951,
                  'precision': 0.

In [32]:
print(np.array(json.loads(report_dict["confusion_matrix"])))

[[10043  2635   928   224   272]
 [ 1991  2504  1399   334   251]
 [  962  1887  3748  1697   787]
 [  431   742  2882  7281  5288]
 [  867   858  2711  8609 40353]]


In [33]:
import json

def calculate_score(cr: dict):
    
    values = []
    values.append(cr["1"]["recall"])
    values.append(cr["2"]["recall"])
    values.append(cr["3"]["recall"])
    values.append(cr["4"]["recall"])
    values.append(cr["5"]["precision"])
    
    mean = 0
    for v in values:
        if v == 0:
            mean = 0
            break
        else:
            mean += 1 / v
    if mean > 0:
        mean = len(values) / mean

    return mean




score = calculate_score(json.loads(report_dict['classification_report']))
print(f'Overall score: {score}')

Overall score: 0.5070364344705827


# Save Report

In [38]:
if os.path.exists(REPORT_FILE):
    report_df = pd.read_csv(REPORT_FILE)
else:
    report_df = pd.DataFrame()
report_df = report_df.append(report_dict, ignore_index=True)
report_df.to_csv(REPORT_FILE, index=False)
report_df

,classification_report,cm_time_min,confusion_matrix,cr_time_min,description,file,library,model_file,model_name,model_save_time_min,pipeline_file,predict_time_min,status,status_date,test_examples,test_features,total_time_min,train_examples,train_features,train_time_min
0,"{""1"": {""precision"": 0.7026024905554779, ""recal...",2.59,"[[10043, 2635, 928, 224, 272], [1991, 2504, 13...",2.71,review_body-tfidf-df_none-ngram13-995688-4073-...,/home/jupyter/dataset/amazon_reviews/amazon_re...,pyspark,../../models/review_body-tfidf-df_none-ngram13...,LogisticRegression,0.04,../../models/review_body-tfidf-df_none-ngram13...,0.0,success,2019-11-21 18:53:49,99684.0,4073.0,7.16,896004.0,4073.0,1.82
